<a href="https://colab.research.google.com/github/achri19/komo_estuary_tutorial/blob/main/0_SetupColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="5">Build Your Own ANUGA Model (BYOAM) </font>

<font size="3">In this notebook, we will:

- Setup Google Colab

- Install packages


- Mount Google Drive </font>


<font size="3">This will require a Google account and you will need to grant access to your Google Drive, where we will store files.</font>



<font size=5> Step #1: Mount Google Drive and Grant Access <font> <br>




<font size=3> When you run the next cell, a pop-up window will appear asking you to grant access to your Google Drive. You must approve or the notebook will not work. <font> <br>


In [ ]:
import sys
import os
path = os.getcwd() + '/'
where_to_mount = '/content/drive/'

from google.colab import drive
drive.mount(where_to_mount, force_remount=True)

mounted_drive = where_to_mount + 'MyDrive/'

Mounted at /content/drive/


There is a folder of necessary files located here: 
https://drive.google.com/drive/folders/1Q0UglZJhOsK2_v7pOuX_FIN60d-gYnM7?usp=sharing

Click on this link and add a SHORTCUT TO DRIVE. This will link this folder to your personal Google Drive and make it accessible to this notebook.

<img src='https://raw.githubusercontent.com/achri19/komo_estuary_tutorial/main/img/Picture1.png?token=GHSAT0AAAAAABZ2GLNI64ZZXQFHCEQJAFU4Y3453YA' width=300px>

<img src='https://raw.githubusercontent.com/achri19/komo_estuary_tutorial/main/img/Picture2.png?token=GHSAT0AAAAAABZ2GLNJGGFPAYAKUN5AIS3EY3454FQ' width=300px>


<font size=5> Step #2: Install packages. <font> <br>
<font size=3> This cell should install all Python packages you need for each tutorial.<font>

In [ ]:
import os
os.chdir(mounted_drive)
if os.path.isdir(mounted_drive + 'komo_estuary_tutorial'):
  print('## Updating the local git repository \n')
  os.chdir(mounted_drive + '/komo_estuary_tutorial')
  ! git pull 
else:
  print('## Pulling the git repository with files for the tutorial\n')
  ! git clone https://github.com/achri19/komo_estuary_tutorial.git

print('\n## Installing the Python packages needed for these tutorials\n')
!/bin/bash $mounted_drive/komo_estuary_tutorial/install_packages.sh


## Updating the local git repository 

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/achri19/komo_estuary_tutorial
   54d793d..54efd8a  main       -> origin/main
Updating 54d793d..54efd8a
Fast-forward
 0_SetupColab.ipynb | 209 +++++++++++++++++++++++------------------------------
 1 file changed, 90 insertions(+), 119 deletions(-)

## Installing the Python packages needed for these tutorials

(1) Install pip packages
nose mpi4py triangle Pmw pymetis cmocean geopandas fiona pygeos rasterio rasterstats
(2) Install gdal
(3) Install netcdf4
(4) Download anuga_core github repository
https://github.com/GeoscienceAustralia/anuga_core
(5) Install anuga
(7) Completed


<font size=5> Step #3: Set up working directories<font> 


<font size=3> We will access data files stored on a shared Google Drive. You will also need to save files to your own Google Drive.<font>



In [ ]:
path_code = where_to_mount + 'MyDrive/processing/code/'
path_templates = where_to_mount + 'MyDrive/processing/templates/'
path_configs = where_to_mount + 'MyDrive/processing/configs/'
path_ancillary = where_to_mount + 'MyDrive/processing/ancillary/'
try:
  os.mkdir(path_examples)
except:''
sys.path.insert(1,path_code)


<font size=5> Step #4: Now we will import Python packages, which will be used for future tutorials. If there are any errors, try rerunning the first cell <font> 




In [ ]:
# Standard packages
import time
import fnmatch
import numpy as np
import scipy
import scipy.interpolate
import pandas as pd
from datetime import datetime
from string import Template
import geopandas as gpd
from shapely.geometry import Polygon, Point
import rasterio
from pathlib import Path
from shapely.geometry import LineString

## ANUGA packages
import anuga
from anuga.utilities.plot_utils import Make_Geotif
from anuga.coordinate_transforms.geo_reference import Geo_reference
from anuga.utilities import animate
from anuga import myid, numprocs, finalize, barrier
from anuga.parallel.parallel_inlet_operator import Parallel_Inlet_operator

## Plotting modules
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

#mpl.rcParams['animation.ffmpeg_path'] = r'/Users/alchrist/Downloads/ffmpeg'
## If you get an error with producing animation plots, uncomment the line above and add directory for ffmpeg (r'/PATH/TO/ffmpeg')
import matplotlib.animation as animation
writer = animation.FFMpegWriter(fps=60)

## Google Earth Packages
import ee
from ee import batch
import geemap


In [ ]:
sys.path.insert(1,path_code)

from BYOM_Utilities_V1 import (build_directory,
                               get_extent_parameters,
                               make_polygons,
                               make_channel_networks,
                               make_watermask,
                               more_opening)

/content/drive/MyDrive/processing/code/raster_bridges.py:39: UserWarning: ShapelyDeprecationWarning
  warnings.warn('ShapelyDeprecationWarning')
/content/drive/MyDrive/processing/code/raster_bridges.py:40: UserWarning: UserWarning
  warnings.warn('UserWarning')


<font size=5> Step #5: The last step is to check your Google Earth Engine account. <font> <br>


<font size=3> You must sign up for a  Google Earth Engine account if you want to use the first notebook. We use GEE to produce water masks of the study area. However, the water masks will be provided, so the GEE notebook is optional. If you'd like to get a Google Earth Engine account (free), follow the instructions here:

https://signup.earthengine.google.com/#!/ </font>
<br>
<font size=3> Then run the next cell and follow the instructions. You'll need to create a GEE project and allow access to Earth Engine Notebook Client. This will generate a verification code, that you need to copy and paste when askeded below. </font>
---




A pop-up window or new browser tab will open after you run the next cell. Make sure to allow access.

![](https://raw.githubusercontent.com/achri19/komo_estuary_tutorial/main/Picture1.png)

![](https://raw.githubusercontent.com/achri19/komo_estuary_tutorial/main/Picture2.png)

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=RM1os_A1OfdVYK8vg8wi3z-RMAzmv9SJ_P3kitUGoEY&tc=2Nv5jO3OaCeMw52Q-0R016wQfNOlT6lgi-ad-aWPVAg&cc=6bjZHukAfTCcOfzTaUCddB6NCZSLR9Y_tFt8TToEFfY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvu0801V_JbmhZHkuijCdCDrqwAd22-DZ9GV6WPmusXnHcW7g6ztbVY

Successfully saved authorization token.


In [ ]:
# Create a map for defining area of interest
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

<font size=5> The next 6 notebooks will walk you through the process of building, running, and analyzing an ANUGA model for the Komo River Estuary </font>

<font size=5 color='red'>You must do the following step to get access to 

To get access to the Komo Tutorial Files, you will need to unzip the komo.zip file from the Github page.

In [ ]:
!ls

0_SetupColab.ipynb	  5_Run_ANUGA.ipynb		komo.zip
1_GEEwatermask.ipynb	  6_Post_Process.ipynb		__MACOSX
2_Setup_DEM.ipynb	  install_packages.sh		mounted_drive
3_Build_ANUGA_Mesh.ipynb  komo				Picture1.png
3_Get_Boundaries.ipynb	  Komo_ANUGA_Instructions.docx	Picture2.png
4_Build_ANUGA_Mesh.ipynb  Komo_ANUGA_Instructions.pdf	README.md


In [ ]:
if os.path.isdir(mounted_drive + 'komo') == False:
  !unzip komo_starter.zip -d $mounted_drive 
